<a href="https://colab.research.google.com/github/digvijaylp/submit/blob/main/SuBMIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SuBMIT: Structure Based Model(s) Input Toolkit
## Package to generate Coarse-Grained Structure (.gro/.pdb) and Topology (.top/.xml) for using Augmented Structure Based Models MD Simulations on GROMACS and OpenSMOG (OpenMM based)
<img src="https://github.com/sglabncbs/submit/blob/main/examples/SuBMIT.png">

In [120]:
#@title Load dependencies:-
import ipywidgets as widgets
from IPython.display import display
from sympy import symbols, cos

In [40]:
#@title Select preset models:-
models={ "custom":"SuBMIT custom SBM Options",\
            "clementi2000":"Clementi et. al. 2000 CA-only model. 10.1006/jmbi.2000.3693",\
	        "afsar2008":"Zarrine-Afsar et. al. 2008 CA-only + hydrophobic model with . 10.1073/pnas.0801874105",\
	        "azia2009":"Azia 2009 CB-CA + Debye-Huckel model. 10.1016/j.jmb.2009.08.010",\
	        "pal2019":"Pal & Levy 2019 Protein CB-CA & RNA/DNA P-S-B model. 10.1371/journal.pcbi.1006768",\
	        "reddy2017":"Reddy & Thirumalai 2017 SOP-SC CA-CB. 10.1021/acs.jpcb.6b13100",\
	        "denesyuk2013":"Denesyuk & Thirumalai 2013 Three Interaction Site TIS P-S-B model. 10.1021/jp401087x",\
	        "chakraborty2018":"Chakraborty & Thirumalai 2018 Three Interaction Site TIS P-S-B model. 10.1021/acs.jctc.8b00091",\
	        "baul2019":"Baul et. al. 2019 SOP-SC-IDP CA-CB. 10.1021/acs.jpcb.9b02575",\
	        "baidya2022":"Baidya & Reddy 2022 SOP-SC-IDP CA-CB. 10.1021/acs.jpclett.2c01972",\
        	"baratam2024":"Baratam & Srivastava 2024 SOP-MULTI CA-CB. 10.1101/2024.04.29.591764",\
        	"sop_idr":"Reddy-Thiruamalai(SOPSC) + Baidya-Reddy(SOPIDP) hybrid CA-CB",\
        	"girirao2016":"Generate multi-basin .top/.xml file",\
        	"banerjee2023":"Banerjee & Gosavi 2023 Self-Peptide model. 10.1021/acs.jpcb.2c05917",\
        	"virusassembly":"Preset for structure based virus assembly (dlprakash 2025)",\
}
selectmodel = widgets.Dropdown(description='Models',options=list(models.keys()),value="custom",disabled=False)
button = widgets.Button(description='Model information',button_style='info',tooltip='model info',icon='info')
def description(btn):print (models[selectmodel.value],flush=True)
button.on_click(description)
display (widgets.HBox([selectmodel,button]))


In [41]:
#@title Coarse-graining level:-
if "custom" in selectmodel.value:
    disable_option=False
    prot_cg = widgets.RadioButtons(options=["1-bead/res CA-only", "2-bead/res CB+CA"],value="2-bead/res CB+CA",description='',disabled=disable_option,)
    nucl_cg = widgets.RadioButtons(options=["1-bead/res P-only", "3-bead/res P-S-B"],value="3-bead/res P-S-B",description='',disabled=disable_option,)
    display (widgets.VBox([widgets.Label('1. Amino-acid CG-level:'),prot_cg]))
    display (widgets.VBox([widgets.Label('2. Nucleotide CG-level'),nucl_cg]))
    #display (widgets.VBox([widgets.Label("1b. Position of C-alpha bead:"),CA_position]))
    #display (widgets.VBox([widgets.Label("1c. Position of C-beta bead:"),CB_position]))
else:
    print ("NOTE: CG paramters are not supported with preset models")
    disable_option=True


In [42]:
#@title Coarse-graining paramters:-
if "custom" in selectmodel.value:
    print ("\n1. Amino-acid bead position and size.")
    CA_position = widgets.Dropdown(options=["CA atom", "Amino-acid backbone COM"],value="CA atom",description='')
    CA_rad = widgets.Textarea(value="1.9",description='')
    display (widgets.HBox([\
          widgets.VBox([widgets.Label("Position of C-alpha bead:"),CA_position]),\
          widgets.VBox([widgets.Label("CA radii (in Å), excl.vol.rad/2:"),CA_rad])\
        ]))
    if prot_cg.value.startswith('2'):
      CB_position = widgets.Dropdown(options=["CB atom", "Amino-acid sidechain COM","Farthest sidechain non-H atom"],value="CB atom",disabled=disable_option,description='')
      CB_rad = widgets.Textarea(value="1.5",description='')
      display (widgets.HBox([\
                widgets.VBox([widgets.Label("Position of C-beta bead:"),CB_position]),\
                widgets.VBox([widgets.Label("CB radii (in Å), excl.vol.rad/2:"),CB_rad])\
             ]))

    print ("\n2. Nucleotide bead position and size.")
    P_position = widgets.Dropdown(description='',options=['P-atom','OP1 atom', 'OP2 atom'],value="P-atom",disabled=False)
    P_rad = widgets.Textarea(value="1.9",description='')
    display (widgets.HBox([\
          widgets.VBox([widgets.Label("Position of P bead:"),P_position]),\
          widgets.VBox([widgets.Label("P radii (in Å), excl.vol.rad/2:"),P_rad])\
      ]))
    if nucl_cg.value.startswith('3'):
      S_position = widgets.Dropdown(description='',options=["C1'-atom","C2'-atom","C3'-atom","C4'-atom","O4'-atom","C5'-atom"],value="C4'-atom",disabled=False)
      S_rad = widgets.Textarea(value="1.9",description='')
      display (widgets.HBox([\
            widgets.VBox([widgets.Label("Position of S bead:"),S_position]),\
            widgets.VBox([widgets.Label("S radii (in Å), excl.vol.rad/2:"),S_rad])\
      ]))
      Bpu_position = widgets.Dropdown(description='',options=["N1","C2","N3","C4","C5","C6","N7","C8","N9","COM"],value="COM",disabled=False)
      Bpy_position = widgets.Dropdown(description='',options=["N1","C2","N3","C4","C5","C6","COM"],value="COM",disabled=False)
      Bpu_rad = widgets.Textarea(value="1.5",description='')
      Bpy_rad = widgets.Textarea(value="1.5",description='')
      display (widgets.HBox([\
          widgets.VBox([widgets.Label("Position of A/G B bead:"),Bpu_position]),\
          widgets.VBox([widgets.Label("A/G B radii (in Å), excl.vol.rad/2:"),Bpu_rad])\
      ]))
      display (widgets.HBox([\
          widgets.VBox([widgets.Label("Position of C/U/T B bead:"),Bpy_position]),\
          widgets.VBox([widgets.Label("C/U/T B radii (in Å), excl.vol.rad/2:"),Bpy_rad])\
      ]))


else:
    print ("NOTE: CG paramters are not supported with preset models")
    disable_option=True



1. Amino-acid bead position and size.



2. Nucleotide bead position and size.


In [152]:
#@title Force-field parameters:-
if "custom" in selectmodel.value:
    Kb,r,r0=symbols('K_b r r_0')
    Ka,t,t0=symbols('K_a theta theta_0')
    Kd,Kdbb,Kdsc,fn,n,p,p0=symbols('K_d K_d^(bb) K_d^(sc) f_n n phi phi_0')
    Kr,Ci,Cj,rij=symbols('K_rep C_i C_j r_ij')
    #theta,phi,C,fn = symbols('r theta phi C fn')

    print ("\n1. Proteins FF force constants.\n\n")
    Kb_prot=widgets.Textarea(value="200.0")
    Ka_prot=widgets.Textarea(value="40.0")
    Kd_bb_prot=widgets.Textarea(value="1.0")
    mulfac_prot=widgets.Textarea(value="2.0")
    Kr_prot=widgets.Textarea(value="1.0")
    #Kd_sc_prot=widgets.Textarea(value="40.0")


    display (0.5*Kb*(r-r0)**2,widgets.HBox([widgets.Label("Bond (Kb in E/Å^2) = "),Kb_prot]))
    print ("\n")
    display (0.5*Ka*(t-t0)**2,widgets.HBox([widgets.Label("Angle (Ka in E/rad^2) = "),Ka_prot]))
    print ("\n")
    display ((Kdbb/fn)*(1-cos(n*(p-p0))),widgets.HBox([widgets.Label("Backbone dihedral (Kd_bb) = "),Kd_bb_prot]))
    display (widgets.HBox([widgets.Label("Kd_bb factor (fn for n=3)) = "),mulfac_prot]))
    print ("\n")
    display ((Kr*(((Ci+Cj)/rij)**12)),widgets.HBox([widgets.Label("Repulsion strength (Krep) = "),Kr_prot]))
    print ("\n")
    #if prot_cg.value.startswith('2'):

    print ("\n1. RNA/DNA FF force constants.\n\n")
    Kb_nucl=widgets.Textarea(value="200.0")
    Ka_nucl=widgets.Textarea(value="40.0")
    Kd_bb_nucl=widgets.Textarea(value="1.0")
    Kd_sc_nucl=widgets.Textarea(value="1.0")
    mulfac_nucl=widgets.Textarea(value="1.0")
    Kr_nucl=widgets.Textarea(value="1.0")
    #Kd_sc_prot=widgets.Textarea(value="40.0")


    display (0.5*Kb*(r-r0)**2,widgets.HBox([widgets.Label("Bond (Kb in E/Å^2) = "),Kb_nucl]))
    print ("\n")
    display (0.5*Ka*(t-t0)**2,widgets.HBox([widgets.Label("Angle (Ka in E/rad^2) = "),Ka_nucl]))
    print ("\n")
    display ((Kd/fn)*(1-cos(n*(p-p0))))
    display(widgets.HBox([widgets.Label("Backbone PPPP dihedral (Kd_bb) = "),Kd_bb_nucl]))
    if nucl_cg.value.startswith('3'):
      display(widgets.HBox([widgets.Label("Sidechain BSSB dihedral (Kd_sc) = "),Kd_bb_nucl]))
    display (widgets.HBox([widgets.Label("Kd_bb/Kd_sc factor (fn for n=3)) = "),mulfac_nucl]))
    print ("\n")
    display ((Kr*(((Ci+Cj)/rij)**12)),widgets.HBox([widgets.Label("Repulsion strength (Krep) = "),Kr_prot]))
    print ("\n")




else:
    print ("NOTE: CG paramters are not supported with preset models")
    disable_option=True



1. Proteins FF force constants.




0.5*K_b*(r - r_0)**2

0.5*K_a*(theta - theta_0)**2

K_d^(bb)*(1 - cos(n*(phi - phi_0)))/f_n

K_rep*(C_i + C_j)**12/r_ij**12




1. RNA/DNA FF force constants.




0.5*K_b*(r - r_0)**2

0.5*K_a*(theta - theta_0)**2

K_d*(1 - cos(n*(phi - phi_0)))/f_n

K_rep*(C_i + C_j)**12/r_ij**12

In [ ]:
"""
    parser.add_argument("--cutoff","-cutoff",type=float,help="User defined Cut-off (in Angstrom) for contact-map generation. Default: 4.5 Å (for all-atom) or 8.0 Å (for coarse-grianed)")
    parser.add_argument("--cutofftype","-cutofftype",type=int,help="-1 No map, 0 use -cmap file, 1 all-atom mapped to CG, 2: coarse-grain . Default: 1")
    parser.add_argument("--W_cont","-W_cont",action="store_true",help="Weight (and normalize) CG contacts based on all atom contact pairs")
    parser.add_argument("--cmap","-cmap",nargs='+',help="User defined cmap in format chain1 atom1 chain2 atom2 weight(opt) distance(opt)")
    parser.add_argument("--scaling","-scaling", help="User defined scaling for mapping to all-atom contact-map.")
    parser.add_argument("--contfunc","-contfunc",type=int,help="1: LJ C6-C12, 2 LJ C10-C12, 3 LJ C12-C18, 6 Gauss + excl, 7 Multi Gauss  . Default: 2")
"""
if "custom" in selectmodel.value:
    disable_option=False
    cutofftype=widgets.Dropdown(options=["-1 No map", "0 User upload file"],value="2-bead/res CB+CA",description='',disabled=disable_option,)
    prot_cg = widgets.RadioButtons(options=["1-bead/res CA-only", "2-bead/res CB+CA"],value="2-bead/res CB+CA",description='',disabled=disable_option,)
    nucl_cg = widgets.RadioButtons(options=["1-bead/res P-only", "3-bead/res P-S-B"],value="3-bead/res P-S-B",description='',disabled=disable_option,)
    display (widgets.VBox([widgets.Label('1. Amino-acid CG-level:'),prot_cg]))
    display (widgets.VBox([widgets.Label('2. Nucleotide CG-level'),nucl_cg]))
    #display (widgets.VBox([widgets.Label("1b. Position of C-alpha bead:"),CA_position]))
    #display (widgets.VBox([widgets.Label("1c. Position of C-beta bead:"),CB_position]))
else:
    print ("NOTE: CG paramters are not supported with preset models")
    disable_option=True